In [1]:
from Query import Query
from Email import Email
from glob import glob
import os
from Tempo import DataHora
from Acesso import Login
import pandas as pd
from datetime import timedelta

data=DataHora()

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [2]:
querys={

    'Estoque':
    
    """

    SELECT * FROM netfeira.vw_stkaryzta

    """,
    
    'TargetEstatico':
    
    """
    
    SELECT * FROM netfeira.vw_targetestatico
    
    """,
    
    'Produto':
    
    """
    
    SELECT * FROM netfeira.vw_produto    
    
    """

}

In [3]:
tabela_df=sql.CriarTabela(kwargs=querys)

In [4]:
tabela_df['TargetEstatico']=tabela_df['TargetEstatico'].loc[(tabela_df['TargetEstatico']['Tipo de Operação']=='VENDAS')]

In [5]:
tabela_df['TargetEstatico'].columns

Index(['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'SKU', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA',
       'Total Geral', 'Total Venda', 'Comissão R$', 'Margem Bruta R$',
       'Cad Vendedor', 'Situação', 'Peso Bruto KG', 'Peso Líquido KG'],
      dtype='object')

In [6]:
tabela_df['Produto'].columns

Index(['SKU', 'Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Grupo de Produtos',
       'Grupo Indústria', 'Grupo de Meta', 'Grupo MCCAIN', 'Linha Fabrica',
       'Grupo de Promotores', 'Peso Liquido', 'Peso Bruto', 'Fator CX',
       'Unid. CMP', 'Unid. STK', 'Fator CMP'],
      dtype='object')

In [7]:
tabela_df['TargetEstatico']=tabela_df['TargetEstatico'].merge(tabela_df['Produto'],on='SKU',how='inner')[['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'SKU','Cód. Fabricante', 'Produto', 'Status', 'Fabricante','Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA',
       'Total Geral', 'Total Venda', 'Comissão R$', 'Margem Bruta R$',
       'Cad Vendedor', 'Situação', 'Peso Bruto KG', 'Peso Líquido KG','Unid. CMP','Fator CMP']]

In [8]:
vendas_df=pd.DataFrame()

vendas_df=tabela_df['TargetEstatico']

In [12]:
dt_max=vendas_df['Data de Faturamento'].max()

dt_min=vendas_df['Data de Faturamento'].max()-timedelta(days=31)

In [14]:
vendas_df=vendas_df.loc[(vendas_df['Data de Faturamento'].between(dt_min,dt_max))]

In [15]:
vendas_df=vendas_df[['Data de Faturamento','SKU','Produto','Unid. CMP','Fator CMP','Qtde']].loc[vendas_df['Fabricante'].str.contains('ARYZTA')].groupby(['Data de Faturamento','SKU','Produto','Unid. CMP','Fator CMP'],as_index=False).sum()

In [16]:
vendas_df

,Data de Faturamento,SKU,Produto,Unid. CMP,Fator CMP,Qtde
0,2022-06-25,9417,PETIT GATEAU CHOCOLATE 2UND DE 85G - 170G PCT ...,CX,20.0,0.0
1,2022-06-25,9432,BOLO FORMIGUEIRO 600G CANDY,CX,6.0,1.0
2,2022-06-25,9433,BOLO CENOURA 600G CANDY,CX,6.0,1.0
3,2022-06-25,9439,"PÃO HB BRIOCHE 4"" CT (15X64G)",CX,8.0,80.0
4,2022-06-25,9440,PÃO HB BRIOCHE MAX CT (12X96G),CX,12.0,68.0
...,...,...,...,...,...,...
449,2022-07-26,9467,CROISSANT MULTIGRAOS PERU/CREAM CHEESE PCT 2 UNID,CX,8.0,10.0
450,2022-07-26,9490,PÃO HB TRADICIONAL PN10 FP ARYZTA (PCT 4UNID ...,CX,21.0,62.0
451,2022-07-26,9491,PÃO HB PN10 TRAD C/GERGELIM ARYZTA FP (PCT 4UN...,CX,21.0,60.0
452,2022-07-26,9492,PÃO HB PN10 AUSTRALIANO FP (PCT 4UNID X 64G),CX,21.0,81.0


In [17]:
valores=[]

for indice,linha in vendas_df.iterrows():
    
    if(linha['Unid. CMP']=='KG'):
    
        res=linha['Qtde']*linha['Fator CMP']
        
        pass
    
    else:
        
        res=int(linha['Qtde']/linha['Fator CMP'])
        
        pass
    
    valores.append(res)
    
    pass

In [18]:
vendas_df['Convertido']=valores

In [19]:
vendas_df=vendas_df[['SKU','Produto','Convertido']].groupby(['SKU','Produto'],as_index=False).mean()

In [20]:
vendas_df['Convertido']=vendas_df.apply(lambda info: int(info['Convertido']),axis=1)

In [21]:
vendas_df=vendas_df[['SKU','Convertido']].loc[vendas_df['Convertido']>0]

In [22]:
vendas_df

,SKU,Convertido
0,9417,1
1,9419,2
2,9420,3
3,9421,2
4,9422,1
5,9423,3
6,9424,12
7,9425,3
8,9430,1
9,9431,3


In [26]:
estoque_df=tabela_df['Estoque']

In [28]:
local=estoque_df['Tipo'].loc[~estoque_df['Local'].str.contains('SUL')].unique().tolist()

In [29]:
    consolidado_df=estoque_df[['SKU','Cód. Fabricante','Produto','Fabricante','Fator CX','Caixa Disponível']].loc[estoque_df['Tipo'].isin(local)].groupby(['SKU','Cód. Fabricante','Produto','Fabricante','Fator CX'],as_index=False).sum()

In [32]:
consolidado_df=consolidado_df.merge(vendas_df,on='SKU',how='left')

In [35]:
consolidado_df.loc[consolidado_df['Convertido'].isnull(),'Convertido']=0

In [42]:
consolidado_df['Dias']=consolidado_df.apply(lambda info: int(info['Caixa Disponível']/info['Convertido']) if info['Convertido']>0 else info['Caixa Disponível'],axis=1)

In [44]:
consolidado_df.rename(columns={'Convertido':'Qtde Média'},inplace=True)

In [45]:
consolidado_df

,SKU,Cód. Fabricante,Produto,Fabricante,Fator CX,Caixa Disponível,Qtde Média,Dias
0,9417,526300016,PETIT GATEAU CHOCOLATE 2UND DE 85G - 170G PCT ...,ARYZTA,20.0,26,1.0,26
1,9419,583800013,PAO ARTESANAL COM AZEITONAS 12UND,ARYZTA,12.0,16,2.0,8
2,9420,525900001,COOKIE BAUNILHA COM GOTAS DE CHOCOLATE 180GR (...,ARYZTA,13.0,15,3.0,5
3,9421,525900003,COOKIE CHOCOLATE COM GOTAS CHOCOLATE 180GR (5 ...,ARYZTA,13.0,8,2.0,4
4,9422,583800012,PAO ARTESANAL BOLA COM CHOCOLATE (12UND X 300G),ARYZTA,12.0,17,1.0,17
5,9423,583800004,PÃO ARTESANAL ITALIANO BOLA MADRE (10UNID X 400G),ARYZTA,10.0,68,3.0,22
6,9424,583800005,PÃO ARTESANAL ITALIANO FILÃO MADRE (10UN X 400G),ARYZTA,10.0,28,12.0,2
7,9425,583800010,PÃO ARTESANAL MULTIGRAOS (16UNID X 300G),ARYZTA,16.0,15,3.0,5
8,9430,503500109,CONGELADO MINI PAO HB CT CX (24X30G),ARYZTA,8.0,44,1.0,44
9,9431,503500265,PAO HB MINI BRIOCHE CT CX (24X30G),ARYZTA,8.0,89,3.0,29
